In [41]:
import numpy as np 
import pandas as pd
import os
from tqdm import tqdm

In [166]:
#os.getcwd()

In [4]:
os.listdir('../Google-Data-411')

['part-01_task_events.csv',
 'part-06_task_events.csv',
 'Week 17 - Coursework-2(1).pdf',
 'part-07_task_events.csv',
 'machine_events.csv',
 'Google cluster-usage traces format schema 2014-11-17 external.pdf',
 'part-04_job_events.csv',
 'part-05_task_usage.csv',
 'part-05_task_events.csv',
 'part-03_job_events.csv',
 'part-02_task_usage.csv',
 'part-03_task_usage.csv',
 'part-02_job_events.csv',
 'part-02_task_events.csv',
 'part-04_task_usage.csv',
 'part-05_job_events.csv',
 'part-06_task_usage.csv',
 'part-07_job_events.csv',
 'part-01_task_usage.csv',
 'part-03_task_events.csv',
 'part-01_job_events.csv',
 'part-04_task_events.csv',
 'part-06_job_events.csv',
 'part-07_task_usage.csv']

In [5]:
job_events = [] # hold all data
task_events = []
task_usage = []

os.chdir('../Google-Data-411') # move to data directory

for file in os.listdir('../Google-Data-411'): # for each file name in the data directory
    if 'job' in file: # if file name has job in it
        job_events.append(pd.read_csv(file, header=None)) # append to job events list
    elif 'task_events' in file:
        task_events.append(pd.read_csv(file, header=None))
    elif 'task_usage' in file:
        task_usage.append(pd.read_csv(file, header=None))

job_events = pd.concat(job_events) # concatenate all this data to be one big dataframe
task_events = pd.concat(task_events) 
task_usage = pd.concat(task_usage)
machine_events = pd.read_csv('machine_events.csv', header=None)

os.chdir('../pre-processing') # move back to working directory

In [6]:
#
# Output size of these dataframes
#

print('Job events dimensions: ', job_events.shape)
print('Task events dimensions: ', task_events.shape)
print('Task usage dimensions: ', task_usage.shape)

Job events dimensions:  (21161, 8)
Task events dimensions:  (1244596, 13)
Task usage dimensions:  (17944928, 20)


## Job Events

In [7]:
#
# Remove NaN column
#

job_events = job_events.drop(columns=1)
job_events.columns = ['timestamp', 'jobID', 'eventType', 'userName', 'schedClass', 'jobName', 'logicalJobName'] # set column names appropriately
job_events.head() # preview data

,timestamp,jobID,eventType,userName,schedClass,jobName,logicalJobName
0,35699140218,6253857847,0,F2+Gv53Pxd4KDRb/UsGECThH/XUOpcWKElUXJkhkt1c=,0,4m7As6A4qnk1L6sLqKjDi5wPRTDuMYAMkRH+JtWXQXs=,j25eTfDZ4FFHzd7p+VKe13tP4+iQbKkHk9VI0SBK/nE=
1,35700609060,6253857847,1,F2+Gv53Pxd4KDRb/UsGECThH/XUOpcWKElUXJkhkt1c=,0,4m7As6A4qnk1L6sLqKjDi5wPRTDuMYAMkRH+JtWXQXs=,j25eTfDZ4FFHzd7p+VKe13tP4+iQbKkHk9VI0SBK/nE=
2,35701412634,6253857893,0,jOqcqHrLBrhAcJPUf2gm/ikKchA7pXWzCssLQqGUSOY=,0,ZzD7sacs6fh4yIPErm24OLwXRCVm1POOzNbZIha+dLU=,PXOVkbAZiqgXwIl/k2Rku4p57uoJn7wn2v3Yfb1OpiA=
3,35701416906,6253856648,4,ZHAzkawqKRJLjH0vstw83yLYltCctQ0DZEmZvQgnTvM=,1,hp6k//YcNIMByCUSSzue2o0b8lD/8m9UwvgMNp+CwRE=,oHQYViMUeNEojiW9p3u0Vdt8N4KBASt2eODnyJhwSSo=
4,35702914162,6253857893,1,jOqcqHrLBrhAcJPUf2gm/ikKchA7pXWzCssLQqGUSOY=,0,ZzD7sacs6fh4yIPErm24OLwXRCVm1POOzNbZIha+dLU=,PXOVkbAZiqgXwIl/k2Rku4p57uoJn7wn2v3Yfb1OpiA=


In [8]:
#
# Factorise userName as numbers, jobName as numbers, logicalJobName as numbers
#
print('# Unique Names: ', len(job_events['userName'].unique()))
print('# Unique Job Names: ', len(job_events['jobName'].unique()))
print('# Unique Logical Job Names: ', len(job_events['logicalJobName'].unique()))

factorised_names, original_names = pd.factorize(job_events['userName'])
factorised_job_names, original_job_names = pd.factorize(job_events['jobName'])
factorised_logical_job_names, original_logical_job_names = pd.factorize(job_events['logicalJobName'])

job_events['userName'] = factorised_names
job_events['jobName'] = factorised_job_names
job_events['logicalJobName'] = factorised_logical_job_names
job_events.head()

# Unique Names:  119
# Unique Job Names:  3500
# Unique Logical Job Names:  2498


,timestamp,jobID,eventType,userName,schedClass,jobName,logicalJobName
0,35699140218,6253857847,0,0,0,0,0
1,35700609060,6253857847,1,0,0,0,0
2,35701412634,6253857893,0,1,0,1,1
3,35701416906,6253856648,4,2,1,2,2
4,35702914162,6253857893,1,1,0,1,1


# Task Events

In [9]:
task_events.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,20646168368,NaN,515042969,17,NaN,5,/fk1fVcVxZ6iM6gHZzqbIyq56m5zrmHfpdcZ/zzkq4c=,2,0,0.01562,0.01553,0.000215,0
1,20646168371,NaN,515042969,17,NaN,0,/fk1fVcVxZ6iM6gHZzqbIyq56m5zrmHfpdcZ/zzkq4c=,2,0,0.01562,0.01553,0.000215,0
2,20646876180,NaN,6252924356,0,NaN,0,r/Al6kYJOwZITr6wi4pAlEwyGv5TM2EkJ8woA5hszeA=,2,8,0.06873,0.04773,0.000038,0
3,20646899409,NaN,515042969,2,NaN,5,/fk1fVcVxZ6iM6gHZzqbIyq56m5zrmHfpdcZ/zzkq4c=,2,0,0.01562,0.01553,0.000215,0
4,20646899412,NaN,515042969,2,NaN,0,/fk1fVcVxZ6iM6gHZzqbIyq56m5zrmHfpdcZ/zzkq4c=,2,0,0.01562,0.01553,0.000215,0


In [10]:
#
# Remove first missing column
#

task_events = task_events.drop(columns=1)
task_events.columns = ['timestamp', 'jobID', 'taskIndex', 'machineID', 'eventType', 'userName', 'schedulingClass', 'priority', 'CPU', 'RAM', 'Disk', 'machineConstraint'] # set column names appropriately
task_events.head() # preview data

,timestamp,jobID,taskIndex,machineID,eventType,userName,schedulingClass,priority,CPU,RAM,Disk,machineConstraint
0,20646168368,515042969,17,NaN,5,/fk1fVcVxZ6iM6gHZzqbIyq56m5zrmHfpdcZ/zzkq4c=,2,0,0.01562,0.01553,0.000215,0
1,20646168371,515042969,17,NaN,0,/fk1fVcVxZ6iM6gHZzqbIyq56m5zrmHfpdcZ/zzkq4c=,2,0,0.01562,0.01553,0.000215,0
2,20646876180,6252924356,0,NaN,0,r/Al6kYJOwZITr6wi4pAlEwyGv5TM2EkJ8woA5hszeA=,2,8,0.06873,0.04773,0.000038,0
3,20646899409,515042969,2,NaN,5,/fk1fVcVxZ6iM6gHZzqbIyq56m5zrmHfpdcZ/zzkq4c=,2,0,0.01562,0.01553,0.000215,0
4,20646899412,515042969,2,NaN,0,/fk1fVcVxZ6iM6gHZzqbIyq56m5zrmHfpdcZ/zzkq4c=,2,0,0.01562,0.01553,0.000215,0


In [11]:
#
# Find number of NaN machine IDs
#
print('Percentage of NaN machine IDs: ', 100*(np.sum(task_events['machineID'].isna())/len(task_events)))

Percentage of NaN machine IDs:  37.01112650209385


### Consider either removing these observations or making them obvious during analysis

In [12]:
#
# Factorise all machine IDs (make note of NaN ID), userName, 
#
factorised_machine_ids, original_machine_ids = pd.factorize(task_events['machineID'])
task_events['machineID'] = factorised_machine_ids
factorised_usernames, original_usernames = pd.factorize(task_events['userName'])
task_events['userName'] = factorised_usernames
task_events.head()

,timestamp,jobID,taskIndex,machineID,eventType,userName,schedulingClass,priority,CPU,RAM,Disk,machineConstraint
0,20646168368,515042969,17,-1,5,0,2,0,0.01562,0.01553,0.000215,0
1,20646168371,515042969,17,-1,0,0,2,0,0.01562,0.01553,0.000215,0
2,20646876180,6252924356,0,-1,0,1,2,8,0.06873,0.04773,0.000038,0
3,20646899409,515042969,2,-1,5,0,2,0,0.01562,0.01553,0.000215,0
4,20646899412,515042969,2,-1,0,0,2,0,0.01562,0.01553,0.000215,0


In [47]:
#
# Create separate dataframe without NaN
#

nan_values = (task_events['machineID'] != -1).values  # boolean vector for NaNs in machine ID column
task_events_NA = task_events
task_events_clean = task_events.iloc[nan_values].reset_index()
task_events_clean = task_events_clean.reset_index().drop(columns=['index', 'level_0'])

In [164]:
#
# Find unique pairs of taskIndex and jobID (only in task_events_clean)
# Quickest way to find unique pairs is to find sum of taskIndex and jobID
# This way takes less than 10 seconds compared to 30 minutes any other way
# Number generated is almost certainly going to be unique to that specific process

task_events_clean['processID'] = (task_events_clean['jobID']+0.01*task_events_clean['taskIndex'])
task_combos, task_combo_unique = pd.factorize(task_events_clean['processID'])
task_events_clean['processID'] = task_combos
        
task_events_clean.head()

,timestamp,jobID,taskIndex,machineID,eventType,userName,schedulingClass,priority,CPU,RAM,Disk,machineConstraint,processID
0,20646899535,6252626850,1115,0,3,2,0,0,0.06873,0.011930,0.000115,0,0
1,20646899762,6252897233,2,1,4,1,0,2,0.06873,0.008774,0.000038,0,1
2,20646899918,6252897233,8,2,4,1,0,2,0.06873,0.008774,0.000038,0,2
3,20647630135,515042969,17,3,1,0,2,0,0.01562,0.015530,0.000215,0,3
4,20647975760,6252897233,6,4,4,1,0,2,0.06873,0.008774,0.000038,0,4


In [162]:
#
# Check number of unique pairs is the same
#
print('# Unique pairs in our method ', len(task_events_clean['processID'].unique()))
print('# Unique pairs using pandas function ', len(task_events_clean[['jobID', 'taskIndex']].drop_duplicates().to_numpy()))

# Unique pairs in our method  266605
# Unique pairs using pandas function  266605


# Task Usage

In [14]:
print(task_usage.shape)
task_usage.head()

(17944928, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,40690000000,40691000000,5567180387,99,1332278,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.0000,0.00000,28.450,0.127900,0,0,0.00000
1,40690000000,40692000000,6114773114,224,351655433,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.0000,0.00000,6.849,0.025740,0,0,0.00000
2,40690000000,40800000000,6221861800,4051,367886060,0.04254,0.0150,0.01524,0.01237,0.01245,0.01688,0.02396,0.000049,0.1855,0.07996,2.719,0.004248,0,0,0.06592
3,40690000000,40695000000,6221861800,8474,351655433,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000129,0.0000,0.00000,22.460,0.100200,0,0,0.00000
4,40690000000,40800000000,6221861800,9655,7311311,0.03577,0.0146,0.01486,0.01216,0.01225,0.01706,0.02106,0.000037,0.1572,0.09546,2.635,0.004522,0,0,0.05725


In [15]:
task_usage.columns = ['start', 'end', 'jobID', 'taskIndex', 'machineID', 'cpuMeanUsage', 'canonicalMemUsage', 'assignedMemUsage', 'unmappedCacheMemUsage', 'totalCacheMemUsage', 'maxMemUsage', 'meanDiskTime', 'meanDiskSpaceUsed', 'cpuMaxUsage', 'maxDiskTime', 'cyclesPerInstruction', 'memAccessPerInstruction', 'samplePortion', 'aggType', 'cpuSampledUsage']

In [16]:
#
# Remove NaN values
#

task_usage = task_usage.dropna()

In [143]:
print(task_usage.shape)
task_usage.head()

(15126583, 21)


,start,end,jobID,taskIndex,machineID,cpuMeanUsage,canonicalMemUsage,assignedMemUsage,unmappedCacheMemUsage,totalCacheMemUsage,...,meanDiskTime,meanDiskSpaceUsed,cpuMaxUsage,maxDiskTime,cyclesPerInstruction,memAccessPerInstruction,samplePortion,aggType,cpuSampledUsage,processID
0,40690000000,40691000000,5567180387,99,1332278,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.00000,0.000000,0.0000,0.00000,28.450,0.127900,0,0,0.00000,0
1,40690000000,40692000000,6114773114,224,351655433,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.00000,0.000000,0.0000,0.00000,6.849,0.025740,0,0,0.00000,0
2,40690000000,40800000000,6221861800,4051,367886060,0.04254,0.0150,0.01524,0.01237,0.01245,...,0.02396,0.000049,0.1855,0.07996,2.719,0.004248,0,0,0.06592,0
3,40690000000,40695000000,6221861800,8474,351655433,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.00000,0.000129,0.0000,0.00000,22.460,0.100200,0,0,0.00000,0
4,40690000000,40800000000,6221861800,9655,7311311,0.03577,0.0146,0.01486,0.01216,0.01225,...,0.02106,0.000037,0.1572,0.09546,2.635,0.004522,0,0,0.05725,0


In [157]:
# # Find unique pairs of taskIndex and jobID (only in task_events_clean)
# Quickest way to find unique pairs is to find sum of 0.01*taskIndex and jobID
# This way takes less than 5 seconds compared to 30 minutes any other way
# Number generated is almost certainly going to be unique to that specific process

task_usage['processID'] = (task_usage['jobID']+0.01*task_usage['taskIndex'])
usage_combos_df, usage_unique_combos = pd.factorize(task_usage['processID'])
task_usage['processID'] = usage_combos_df
        
task_usage.head()

,start,end,jobID,taskIndex,machineID,cpuMeanUsage,canonicalMemUsage,assignedMemUsage,unmappedCacheMemUsage,totalCacheMemUsage,...,meanDiskTime,meanDiskSpaceUsed,cpuMaxUsage,maxDiskTime,cyclesPerInstruction,memAccessPerInstruction,samplePortion,aggType,cpuSampledUsage,processID
0,40690000000,40691000000,5567180387,99,1332278,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.00000,0.000000,0.0000,0.00000,28.450,0.127900,0,0,0.00000,0
1,40690000000,40692000000,6114773114,224,351655433,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.00000,0.000000,0.0000,0.00000,6.849,0.025740,0,0,0.00000,1
2,40690000000,40800000000,6221861800,4051,367886060,0.04254,0.0150,0.01524,0.01237,0.01245,...,0.02396,0.000049,0.1855,0.07996,2.719,0.004248,0,0,0.06592,2
3,40690000000,40695000000,6221861800,8474,351655433,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.00000,0.000129,0.0000,0.00000,22.460,0.100200,0,0,0.00000,3
4,40690000000,40800000000,6221861800,9655,7311311,0.03577,0.0146,0.01486,0.01216,0.01225,...,0.02106,0.000037,0.1572,0.09546,2.635,0.004522,0,0,0.05725,4


In [158]:
#
# Check number of unique pairs is the same
#
print('# Unique pairs in our method ', len(task_usage['processID'].unique()))
print('# Unique pairs using pandas function ', len(task_usage[['jobID', 'taskIndex']].drop_duplicates().to_numpy()))

# Unique pairs in our method  301578
# Unique pairs using pandas function  301578


# Machine Events

In [18]:
machine_events.columns = ['timestamp', 'machineID', 'eventType', 'platformID', 'capacityCPU', 'capacityMem']

# Factorise platformID
machine_events['platformID'] = pd.factorize(machine_events['platformID'])[0]
machine_events.head()

,timestamp,machineID,eventType,platformID,capacityCPU,capacityMem
0,0,5,0,0,0.5,0.2493
1,0,6,0,0,0.5,0.2493
2,0,7,0,0,0.5,0.2493
3,0,10,0,0,0.5,0.2493
4,0,13,0,0,0.5,0.2493


# Output

In [165]:
job_events.to_csv('job-events.csv')
task_events_clean.to_csv('task-events.csv')
task_usage.to_csv('task-usage.csv')
machine_events.to_csv('machine_events.csv')